In [1]:
import pyrealsense2 as rs
import matplotlib as plt
import cv2
import logging
import numpy as np
import time
import argparse
import glob

In [29]:
ctx = rs.context()
devices = ctx.query_devices()
var = [i.get_info(rs.camera_info.serial_number) for i in devices]

#set up master and slave
dev = devices[0].first_depth_sensor()
dev.set_option(rs.option.inter_cam_sync_mode,1)
dev.get_option(rs.option.inter_cam_sync_mode)

dev2 = devices[1].first_depth_sensor()
dev2.set_option(rs.option.inter_cam_sync_mode,2)
dev2.get_option(rs.option.inter_cam_sync_mode)

#start pipeline
#cam1: configure depth and color streams
pipeline1 = rs.pipeline()
config1 = rs.config()
config1.enable_device(var[0])
config1.enable_stream(rs.stream.depth, 848, 480, rs.format.z16, 90)
config1.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 60)
#config1.enable_record_to_file('master.bag')
print('cam1 enabled')

#cam2: configure depth and color streams 
pipeline2 = rs.pipeline()
config2 = rs.config()
config2.enable_device(var[1])
config2.enable_stream(rs.stream.depth, 848, 480, rs.format.z16, 90)
config2.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 60)
#config2.enable_record_to_file('slave.bag')
print('cam2 enabled')
prof = pipeline1.start(config1)
prof2 = pipeline2.start(config2)
print('cams configued')

cam1 enabled
cam2 enabled
cams configued


In [30]:
#take checkerboard images
try:
    start = time.time()
    mastercount = []
    slavecount = []
    d = 1
    while time.time() - start <100:

        #camera 1
        frames1 = pipeline1.wait_for_frames()
        depthframe1 = frames1.get_depth_frame()
        colframe1 = frames1.get_color_frame()
        if not depthframe1 or not colframe1:
            continue


        #convert images to numpy arrays
        depthim1 = np.asanyarray(depthframe1.get_data())
        colim1 = np.asanyarray(colframe1.get_data())

        #apply color map on depth image, convert to 8bit pixel
        depthcolmap1 = cv2.applyColorMap(cv2.convertScaleAbs(depthim1, alpha=0.5), cv2.COLORMAP_JET)

        #camera 2
        frames2 = pipeline2.wait_for_frames()
        depthframe2 = frames2.get_depth_frame()
        colframe2 = frames2.get_color_frame()
        if not depthframe2 or not colframe2:
            continue


        #check time sync
        #print([depthframe1, depthframe2]) #returns system_time
        #counter = depthframe1.get_frame_metadata(rs.frame_metadata_value.frame_counter)
        #mastercount.append(counter)
        #counter2 = depthframe2.get_frame_metadata(rs.frame_metadata_value.frame_counter)
        #slavecount.append(counter2)
        #timestamp1 = depthframe1.get_timestamp()
        #timestamp2 = depthframe2.get_timestamp()
        #print([counter,counter2, timestamp1, timestamp2])
        #convert images to numpy arrays
        depthim2 = np.asanyarray(depthframe2.get_data())
        colim2 = np.asanyarray(colframe2.get_data())

        #apply color map on depth image, convert to 8bit pixel
        depthcolmap2 = cv2.applyColorMap(cv2.convertScaleAbs(depthim2, alpha=0.5), cv2.COLORMAP_JET)


        #stack all images horizontally
        images = np.hstack((colim1, depthcolmap1, colim2, depthcolmap2))

        #show images from cameras
        cv2.namedWindow('RealSense', cv2.WINDOW_NORMAL)
        cv2.imshow('RealSense', images)
        cv2.waitKey(1)
        
        #save images and depth map from both cameras by pressing "s"
        ch = cv2.waitKey(25)
        if ch==115:
            filename = "checkerboard/master_%d.jpg"%d
            filename2 = "checkerboard/slave_%d.jpg"%d
            cv2.imwrite(filename, colim1)
            cv2.imwrite(filename2, colim2)
            d+=1
            #cv2.imwrite("depth1.jpg", depthcolmap1)
            #cv2.imwrite("depth2.jpg", depthcolmap2)

finally:
    #stop streaming
    pipeline1.stop()
    pipeline2.stop()
    cv2.destroyAllWindows()
    del pipeline1
    del config1
    del pipeline2
    del config2

In [43]:
#calculate extrinsic and intrinsic params from above checkerboards
#currently for camera1
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
#prepare object points
objp = np.zeros((7*9,3),np.float32)
objp[:,:2] = np.mgrid[0:7,0:9].T.reshape(-1,2)

#arrays to store object points and image points from all images
objpoints = []
masterimgpoints = []

path = '/home/alissa/IntelRealSense/checkerboard/'
images = glob.glob(path + '/master_*.jpg')

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #find chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (7,9),None)
    #if found add object points, image points
    if ret == True:
        objpoints.append(objp)
        corners2 = cv2.cornerSubPix(gray, corners, (11,11),(-1,-1), criteria)
        masterimgpoints.append(corners2)
        #draw and display the corners
        img = cv2.drawChessboardCorners(img, (7,9), corners2, ret)
        cv2.imshow('img', img)
        cv2.waitKey(500)
cv2.destroyAllWindows()


In [44]:
#returns camera matrix, distortion coefficients, rotation, and translation vectors
ret1, mtx1, dist1, rvecs1, tvecs1 = cv2.calibrateCamera(objpoints, masterimgpoints, gray.shape[::-1], None, None)

In [46]:
#calculate extrinsic and intrinsic params from above checkerboards
#currently for one camera
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
#prepare object points
objp = np.zeros((7*9,3),np.float32)
objp[:,:2] = np.mgrid[0:7,0:9].T.reshape(-1,2)

#arrays to store object points and image points from all images
objpoints = []
slaveimgpoints = []

path = '/home/alissa/IntelRealSense/checkerboard/'
images = glob.glob(path + '/slave_*.jpg')

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #find chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (7,9),None)
    #if found add object points, image points
    if ret == True:
        objpoints.append(objp)
        corners2 = cv2.cornerSubPix(gray, corners, (11,11),(-1,-1), criteria)
        slaveimgpoints.append(corners2)
        #draw and display the corners
        img = cv2.drawChessboardCorners(img, (7,9), corners2, ret)
        cv2.imshow('img', img)
        cv2.waitKey(500)
cv2.destroyAllWindows()


In [47]:
#returns camera matrix, distortion coefficients, rotation, and translation vectors
ret2, mtx2, dist2, rvecs2, tvecs2 = cv2.calibrateCamera(objpoints, slaveimgpoints, gray.shape[::-1], None, None)

In [75]:
gray.shape[::-1]

(640, 480)

In [76]:
#K = camera matrix (mtx)
#D = dist (dist)
#F = fundamental matrix
#E = essential matrix
#R = rotation from left to right camera
#T = translation from left to right camera
ret, M1, d1, M2, d2, R, T, E, F = cv2.stereoCalibrate(
    objpoints, masterimgpoints, slaveimgpoints,
    mtx1, dist1, mtx2, dist2, gray.shape[::-1])

In [79]:
mtx1

array([[532.72339941,   0.        , 466.42753055],
       [  0.        , 510.36751616, 243.05093151],
       [  0.        ,   0.        ,   1.        ]])

In [78]:
print('intrinsics matrix master', M1)
print('dist 1', d1)
print('intrinsic matrix slave', M2)
print('dist 2', d2)
print('R',R)
print('T',T)
print('E',E)
print('F',F)

intrinsics matrix master [[532.72339941   0.         466.42753055]
 [  0.         510.36751616 243.05093151]
 [  0.           0.           1.        ]]
dist 1 [[-0.16851522  1.30677432  0.01250941  0.07521741 -4.6680607 ]]
intrinsic matrix slave [[491.37667796   0.         249.34152208]
 [  0.         485.9073681  243.39646834]
 [  0.           0.           1.        ]]
dist 2 [[-1.31199620e-02  4.62466387e-03 -6.94107401e-04 -4.68118632e-02
  -2.07436504e+00]]
R [[-0.17531009  0.36394608 -0.91477299]
 [-0.11342419  0.91550939  0.38597604]
 [ 0.97795773  0.17142288 -0.11921775]]
T [[16.61098535]
 [-9.71021521]
 [20.73837998]]
E [[ -7.1439461  -20.65073472  -6.84688785]
 [-19.88048886   4.70014911 -16.99058561]
 [ -3.58638628  18.74150785  -2.47120022]]
F [[ 2.26795461e-06  6.84306153e-06 -1.56309530e-03]
 [ 6.38240418e-06 -1.57502564e-06  3.11693440e-04]
 [-1.55949150e-03 -4.37454867e-03  1.00000000e+00]]


In [ ]:
cv2.stereoRectify